# Automated_optimization_testing_Sartorious

Importing relevant packages

In [ ]:
#Import relevant python packages
#%% General Imports
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

# plotting dependencies
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
"C:\Users\admin\Documents\GitHub\viscosity_liquid_transfer_Pablo\Sartorious_experiments\Code\Automated_oprimization_testing_Sartorious.ipynb"

Robot initialization and checks

In [ ]:
#Import robot related packages and run setup
import pandas as pd
import time
from matplotlib import pyplot as plt
from pathlib import Path
import sys
REPOS = 'GitHub'
ROOT = str(Path().absolute()).split(REPOS)[0]
sys.path.append(f'{ROOT}{REPOS}')

from polylectric.configs.SynthesisB1 import SETUP, LAYOUT_FILE

from controllably import load_deck      # optional
load_deck(SETUP.setup, LAYOUT_FILE)     # optional

platform = SETUP
platform.mover.verbose = False #askpablo

In [ ]:
#Initialization of variables for platform objects
pipette= platform.setup
deck = platform.setup.deck
balance = platform.balance
balance_deck = deck.slots['1']
source = deck.slots['2']
tip_rack = deck.slots['3']
bin = deck.slots['4']
pipette.mover.setSpeed(50)
print(balance_deck)
print(source)
print(tip_rack)
print(bin)

In [ ]:
#Check if balance is connected
balance.zero() #to tare
balance.toggleRecord(True) # turn on and record weight
time.sleep(5) # do previous action for 5s
print(balance.buffer_df.iloc[-1]) #iloc can take -1, loc needs to be 839 loc[839,["Time","Value","Factor","Baseline","Mass"]]. -1 is last line. to find number of last line, print(balance.buffer_df)
balance.toggleRecord(False) #turn off

In [ ]:
#Establish initial height of liquid on the source vial
pipette_name = 'rLine1000'
initial_liquid_level = 12 # in mm


Importing files + getting parameters

In [ ]:
# Change according to experiment
std = "1275"
liquid_name = 'Viscosity_std_' + std 
density = 0.8736
# Import initialisation file?
REPO = 'viscosity_liquid_transfer_Pablo'
folder = os.getcwd().split(REPO)[0]+REPO+r'\Sartorious_experiments' #folder that data is saved to
# df_para = pd.read_csv(folder+r'\Cleaned_Data\Viscosity_std_'+std+'_modified.csv')
df_para = pd.read_csv(folder+r'\Cleaned_Data\1275_test_df.csv') #importing data saved in folder

#REPOS = 'GitHub'
#ROOT = str(Path().absolute()).split(REPOS)[0] # this => sys.path = os.getcwd().split(REPO)[0]+REPO #askpablo
#sys.path.append(f'{ROOT}{REPOS}') # diff between sys and os? #askpablo

In [ ]:
df_para

In [ ]:
liquids_dict = {
  liquid_name :{ # why so many dict #askpablo
        "rLine1000": {
            "aspiration_rate": float(df_para.loc[1,['aspiration_rate']]), # take number from previous dataset and convert it into float (not integer, we want the decimal places)
            "dispense_rate": float(df_para.loc[1,['dispense_rate']]), # take number from previous dataset and convert it into float (not integer, we want the decimal places)
            "blow_out" : False, # parameters for new expt
            "delay_aspirate" : 10, 
            "delay_dispense" : 10, 
            "delay_blow_out" : 0, 
            },
    }

}

Running the experiment

In [ ]:
def loop_pipetting(iterations, volume, liquid_level, df):
    import pandas as pd
    counter = 1
    pipette.mover.setHandedness(False) # to fix an arm setting

    while counter <= iterations:
        #Transfer start
        start = time.time() # green time = package to handle time #yellow time = records the current PC time

        #Aspirate step
        pipette.mover.safeMoveTo(source.wells['A1'].from_bottom((0,0,liquid_level-5))) #source=2nd plate #from_ must have coordinates, x,y,z, liquid level -3 not as good as -5 (-5 to imemerse tip)
        pipette.liquid.aspirate(volume, speed=liquids_dict[liquid_name][pipette_name]['aspiration_rate'])
        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_aspirate'])

        pipette.touchTip(source.wells['A1']) #LRUPDOWN - happens after every aspiration and blow out

        #Dispense step
        pipette.mover.safeMoveTo(balance_deck.wells['A1'].from_top((0,0,-5))) #balance_deck= 1st plate #-5 goes abit more into the well

        balance.tare() 
        balance.clearCache() #clear balance memory
        balance.toggleRecord(True) #turn on the balance
        time.sleep(5) #stabilise the balance for 5s


        pipette.liquid.dispense(volume, speed=liquids_dict[liquid_name][pipette_name]['dispense_rate'])

        time.sleep(liquids_dict[liquid_name][pipette_name]['delay_dispense'])

        #Blowout step
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True: #do only if blowout right after dispensing is true
            pipette.liquid.blowout(home=False)
            time.sleep(liquids_dict[liquid_name][pipette_name]['delay_blow_out']) #delay time after doing blowout right after dispensing

        #Transfer termination
        finish = time.time() #yellow time takes pc time, green time is the package
        time_m = finish - start

        pipette.mover.safeMoveTo(source.wells['A1'].top) #move pipette to sample retrieval position for next aspiration/blowout
        time.sleep(5)
        balance.toggleRecord(False) #only turns off the balance recording after pipette finished dispensing
        if liquids_dict[liquid_name][pipette_name]['blow_out'] == True:
            pipette.liquid.home() #pipette plunger totally not depressed :)

        #Do blowout
        pipette.liquid.blowout(home=False) #askpablo
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()
        time.sleep(5)
        pipette.liquid.blowout(home=False)
        time.sleep(5)
        pipette.touchTip(source.wells['A1'])
        pipette.liquid.home()

        #Record transfer values 
        m = (balance.buffer_df.iloc[-10:,-1].mean()-balance.buffer_df.iloc[:10,-1].mean())/1000 #-1: mass column #-10: = last 10 rows #:10 = first 10 rows

        #Update dataframe
        df = pd.concat([df,pd.DataFrame(liquids_dict[liquid_name][pipette_name],index=[0])],ignore_index=True) #askpablo index[0], #ignoreindex is to avoid creating additional columns
        error = (m-density*volume/1000)/(density/1000*volume)*100
        df.iloc[-1,-3] = time_m #-1: last row, #-3: 3rd last
        df.iloc[-1,2] = volume
        df.iloc[-1, 0] = liquid_name
        df.iloc[-1, 1] = pipette_name
        df.iloc[-1,-4] = density
        df.iloc[-1, -2] = m
        df.iloc[-1,-1]= error

        #Check liquid levels
        liquid_level = liquid_level - 1.2*m/density
        print("LIQUID LEVEL IS " +str(liquid_level))
        print("LIQUID CHANGE IS " +str(1.2*m/density))
        

        #Liquid level checks
        if (1.2*m/density > 1.2) or (1.2*m/density < 0):
            break
        if (liquid_level > initial_liquid_level) or (liquid_level < 6):
            break

        # Increment Tries
        counter += 1

    return df, liquid_level, 1.2*m/density

In [ ]:
#Check if new tip is required
pipette.mover.setSpeed(50)
pipette.mover.setHandedness(False)

if pipette.liquid.isTipOn()== False:
    pipette.attachTip()

#setup for for loop
volumes_list = [1000, 500, 300]
liquid_level = initial_liquid_level

#New dataframe
# df = pd.DataFrame(columns = ['liquid', 'pipette', 'volume', 'aspiration_rate', 'dispense_rate','blow_out', 'delay_aspirate', 'delay_dispense','delay_blow_out', 'density', 'time', 'm', '%error'])
# df = df.astype({'liquid':str,'pipette':str,'blow_out':bool})

# #Imported dataframe
df = pd.read_csv(liquid_name+'_optimization_exp3.csv')

for item in volumes_list:
    #run liquid transfer function
    df, final_ll, ll_change = loop_pipetting(iterations=6, volume=item, liquid_level=liquid_level, df=df)
    print(df)
    print(final_ll)
    print(ll_change)
    if (ll_change > 1.2) or (ll_change < 0):
        break
    if (final_ll > initial_liquid_level) or (final_ll < 6):
        break




In [ ]:
df

In [ ]:
#save after each standard-experiment iteration
df.to_csv(liquid_name+'_optimization_exp3.csv', index = False)

In [ ]:
df = df.sort_index()

In [ ]:
pipette.ejectTipAt(bin.wells['A1'].top)
pipette.mover.home()

In [ ]:
df

In [ ]:
pipette.mover.home()